# Importing Required Libs

In [24]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



print('Libraries imported.')
from bs4 import BeautifulSoup
import urllib.request
import requests

Libraries imported.


In [ ]:

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
v 

# Assignment part 1:

# Parsing the Wiki Page to repare the required data

In [174]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [175]:
page = urllib.request.urlopen(url)

In [176]:
soup = BeautifulSoup(page, "lxml")

In [177]:
my_table = soup.find('table',{'class':'wikitable sortable'})

# Tranform the data into a *pandas* dataframe

In [178]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

In [179]:
neighborhoods = pd.DataFrame(columns=column_names)

In [180]:
A=[]
B=[]
C=[]



for row in my_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        
     

In [181]:
df=pd.DataFrame(A,columns=['postal_code'])
df['borough']=B
df['neighborhood']=C

df.head()

,postal_code,borough,neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [182]:
df = df[df.borough != 'Not assigned']

In [183]:
df = df[df.neighborhood != 'Not assigned']

In [184]:
df = df.groupby(['postal_code','borough'])['neighborhood'].apply(', '.join).reset_index()

In [185]:
df.head()

,postal_code,borough,neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae\n


# This is the end of part 1 of  the assignment

# Part 2

## Use geopy library to get the latitude and longitude values of New York City.

In [20]:

!pip install geocoder

     |████████████████████████████████| 102kB 14.6MB/s ta 0:00:01


In [186]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    

In [187]:
get_latlng('M4G')

[43.70949500000006, -79.36398897099997]

In [188]:
postal_codes = df['postal_code']    
coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]

In [189]:

df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [190]:

df.head()

,postal_code,borough,neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae\n,43.769688,-79.239440


# this is the end of part 2

# part 3 : 

### lets get the number of boroughs and neighborhoods 

In [191]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [47]:

import requests
from bs4 import BeautifulSoup as bs
import re

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

### creating a map for all Toronto

In [192]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['borough'], df['neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)  
    


map_toronto

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### We will segment and cluster only the neighborhoods in North York.
### So let's slice the original dataframe and create a new dataframe of the North York data.


In [193]:
northyork = df[df['borough'] == 'North York'].reset_index(drop=True)
northyork.head()

,postal_code,borough,neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.802845,-79.356207
1,M2J,North York,"Fairview\n, Henry Farm, Oriole\n",43.780970,-79.347813
2,M2K,North York,Bayview Village,43.781015,-79.380529
3,M2L,North York,"Silver Hills\n, York Mills",43.757095,-79.380320
4,M2M,North York,"Newtonbrook, Willowdale",43.791475,-79.413605


In [194]:
northyork.shape

(24, 5)

Let's get the geographical coordinates of North York.

In [195]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7708175, -79.4132998.


##### As we did with all of Toronto, let's visualizat North York neighborhoods.

In [196]:
# create map of Manhattan using latitude and longitude values
northyork_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(northyork['Latitude'], northyork['Longitude'], northyork['neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(northyork_map)  
    
northyork_map

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [197]:
CLIENT_ID = 'PMPYAEALNW1I0UF4TE0LGY20N4QEMRYFB1SLNDTTQJMEZUHU' # your Foursquare ID
CLIENT_SECRET = 'KRVTAMCWJAJYOAAA42WOBLHWOBYVU2SLLPYPMWGY3TJQXPQI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PMPYAEALNW1I0UF4TE0LGY20N4QEMRYFB1SLNDTTQJMEZUHU
CLIENT_SECRET:KRVTAMCWJAJYOAAA42WOBLHWOBYVU2SLLPYPMWGY3TJQXPQI


## Explore Neighborhoods in North York

In [198]:
LIMIT = 100 
radius = 500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [199]:
northyork_venues = getNearbyVenues(names=northyork['neighborhood'],
                                   latitudes=northyork['Latitude'],
                                   longitudes=northyork['Longitude']
                                  )

Hillcrest Village
Fairview
, Henry Farm, Oriole

Bayview Village
Silver Hills
, York Mills
Newtonbrook, Willowdale
Willowdale South

York Mills West

Willowdale West
Parkwoods
Don Mills North

Flemingdon Park, Don Mills South

Bathurst Manor, Downsview North
, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East

Downsview West
Downsview Central

Downsview Northwest

Victoria Village
Bedford Park, Lawrence Manor East

Lawrence Heights, Lawrence Manor
Glencairn

Downsview, North Park
, Upwood Park

Humber Summit
Emery, Humberlea


In [200]:
northyork_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.802845,-79.356207,Woodbrooke Estate,43.802067,-79.354347,Residential Building (Apartment / Condo)
1,Hillcrest Village,43.802845,-79.356207,Duncan Creek Park,43.805539,-79.360695,Dog Run
2,"Fairview\n, Henry Farm, Oriole\n",43.780970,-79.347813,The LEGO Store,43.778207,-79.343483,Toy / Game Store
3,"Fairview\n, Henry Farm, Oriole\n",43.780970,-79.347813,SilverCity Fairview Mall Cinemas,43.778681,-79.344085,Movie Theater
4,"Fairview\n, Henry Farm, Oriole\n",43.780970,-79.347813,CF Fairview Mall,43.777994,-79.343665,Shopping Mall


## Analyze Each Neighborhood

In [99]:
northyork_split = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_split['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_split.columns[-1]] + list(northyork_split.columns[:-1])
northyork_split = northyork_split[fixed_columns]

northyork_split.head()

,Neighborhood,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Garage,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bookstore,Boutique,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gift Shop,Golf Driving Range,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hockey Arena,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Nightclub,Optical Shop,Park,Pet Store,Pharmacy,Pizza Place,Platform,Plaza,Pub,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Club,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Fairview\n, Henry Farm, Oriole\n",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,"Fairview\n, Henry Farm, Oriole\n",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Fairview\n, Henry Farm, Oriole\n",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [100]:
northyork_grouped = northyork_split.groupby('Neighborhood').mean().reset_index()
northyork_grouped

,Neighborhood,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Garage,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bookstore,Boutique,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gift Shop,Golf Driving Range,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hockey Arena,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Nightclub,Optical Shop,Park,Pet Store,Pharmacy,Pizza Place,Platform,Plaza,Pub,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Club,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Bayview Village,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000
1,"Bedford Park, Lawrence Manor East\n",0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.105263,0.052632,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.052632,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.105263,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.0,0.052632,0.052632,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.052632,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,"CFB Toronto, Downsview East\n",0.25,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000

### new dataframe and display the top 10 venues for each neighborhood.

In [104]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [124]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bayview Village,Construction & Landscaping,Park,Trail,Dog Run,Golf Driving Range,Food Court,Deli / Bodega,Department Store,Dessert Shop,Discount Store
1,"Bedford Park, Lawrence Manor East\n",Italian Restaurant,Coffee Shop,Butcher,Fast Food Restaurant,Liquor Store,Sports Club,Juice Bar,Café,Pharmacy,Sushi Restaurant
2,"CFB Toronto, Downsview East\n",Airport,Park,Coffee Shop,Food Court,Fried Chicken Joint,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant
3,Don Mills North\n,Burger Joint,Park,Women's Store,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Falafel Restaurant
4,Downsview Northwest\n,Discount Store,Pizza Place,Fast Food Restaurant,Grocery Store,Shopping Mall,Vietnamese Restaurant,Gym / Fitness Center,Mobile Phone Shop,Coffee Shop,Sandwich Place


## Cluster Neighborhoods

In [122]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 4, 0, 0, 1, 0, 0, 0], dtype=int32)

In [149]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Bayview Village,Construction & Landscaping,Park,Trail,Dog Run,Golf Driving Range,Food Court,Deli / Bodega,Department Store,Dessert Shop,Discount Store
1,0,"Bedford Park, Lawrence Manor East\n",Italian Restaurant,Coffee Shop,Butcher,Fast Food Restaurant,Liquor Store,Sports Club,Juice Bar,Café,Pharmacy,Sushi Restaurant
2,0,"CFB Toronto, Downsview East\n",Airport,Park,Coffee Shop,Food Court,Fried Chicken Joint,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant
3,4,Don Mills North\n,Burger Joint,Park,Women's Store,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Falafel Restaurant
4,0,Downsview Northwest\n,Discount Store,Pizza Place,Fast Food Restaurant,Grocery Store,Shopping Mall,Vietnamese Restaurant,Gym / Fitness Center,Mobile Phone Shop,Coffee Shop,Sandwich Place


In [201]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

northyork_merged = northyork

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='neighborhood')

northyork_merged = northyork_merged[northyork_merged['Cluster Labels'] != np.nan]


# simply drop whole row with NaN in "price" column
northyork_merged.dropna(subset=["Cluster Labels"], axis=0, inplace=True)

# reset index, because we droped two rows
northyork_merged.reset_index(drop=True, inplace=True)



northyork_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

In [202]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['neighborhood'], northyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1

In [203]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 0, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Women's Store,Food Court,Toy / Game Store,Japanese Restaurant,Tea Room,Baseball Field,Luggage Store
2,North York,0.0,Construction & Landscaping,Park,Trail,Dog Run,Golf Driving Range,Food Court,Deli / Bodega,Department Store,Dessert Shop,Discount Store
3,North York,0.0,Korean Restaurant,Café,Middle Eastern Restaurant,Coffee Shop,Shopping Mall,Halal Restaurant,Grocery Store,Japanese Restaurant,Fried Chicken Joint,Fast Food Restaurant
4,North York,0.0,Coffee Shop,Café,Ramen Restaurant,Fast Food Restaurant,Discount Store,Plaza,Pizza Place,Pharmacy,Movie Theater,Middle Eastern Restaurant
5,North York,0.0,Convenience Store,Speakeasy,Park,Bank,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant
6,North York,0.0,Convenience Store,Intersection,Park,Bakery,Coffee Shop,Eastern European Restaurant,Bus Line,Fried Chicken Joint,Dessert Shop,Discount Store
9,North York,0.0,Bubble Tea Shop,Coffee Shop,Beer Store,Supermarket,Grocery Store,Gym,Intersection,Dog Run,Eastern European Restaurant,Fried Chicken Joint
10,North York,0.0,Furniture / Home Store,Bar,Coffee Shop,Falafel Restaurant,Fast Food Restaurant,Caribbean Restaurant,Massage Studio,Pizza Place,Bank,Japanese Restaurant
11,North York,0.0,Airport,Park,Coffee Shop,Food Court,Fried Chicken Joint,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant
12,North York,0.0,Pizza Place,Beer Store,Hockey Arena,Fast Food Restaurant,Electronics Store,Discount Store,Convenience Store,Coffee Shop,Clothing Store,Spa


#### Cluster 2

In [169]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 1, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,North York,1.0,Park,Food & Drink Shop,Women's Store,Cosmetics Shop,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store
18,North York,1.0,Park,Bakery,Basketball Court,Women's Store,Frozen Yogurt Shop,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store


#### Cluster 3

In [170]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 2, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,North York,2.0,Park,Grocery Store,Women's Store,Fried Chicken Joint,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store


#### Cluster 4

In [171]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 3, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3.0,Residential Building (Apartment / Condo),Dog Run,Women's Store,Fried Chicken Joint,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Eastern European Restaurant,Electronics Store


#### Cluster 5

In [172]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 4, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,North York,4.0,Burger Joint,Park,Women's Store,Department Store,Dessert Shop,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Falafel Restaurant
